In [ ]:
!pip install tensorflow
!pip install keras

In [2]:
from keras import models
from keras import layers
from keras import activations
from keras import initializers
from keras import regularizers
from keras import optimizers
from keras import losses
from keras import metrics
from keras import callbacks
from keras.preprocessing import image

In [3]:
model = models.Sequential()

model.add(layers.Conv2D(
  32, (5, 5), # 32: numero de filtros e (5,5) é o shape do kernel que vai ser passado pela imagem, ela perde n-1 do tamanho em cada dimensão
  input_shape = (64, 64, 3), # 64: tamanho da imagem, 3: diz que a imagem é RGB, se for preto e branco é 1
  activation = 'relu'
))

model.add(layers.MaxPooling2D(
  pool_size = (2, 2) # reduz o tamanho da imagem pela metade, passa um kernel 2x2 e retorna um valor 1x1
))

model.add(layers.Conv2D(
  16, (5, 5),
  input_shape = (30, 30, 3),  # 64 - 4 = 60 - 30 do MaxPooling = 30, 30
  activation = 'relu'
))

model.add(layers.MaxPooling2D(
  pool_size = (2, 2)
))

model.add(layers.Conv2D(
  4, (5, 5),
  input_shape = (13, 13, 3),
  activation = 'relu'
))

model.add(layers.MaxPooling2D(
  pool_size = (2, 2)
))

model.add(layers.Flatten()) # transforma a imagem de matriz 2d para vetor

model.add(layers.Dense(256, # quantidade de neurônios
  kernel_regularizer = regularizers.L2(1e-4), # penaliza pesos altos
  kernel_initializer = initializers.RandomNormal(stddev=1), # inicialização dos pesos, inicia com desvio-padrão de 1, tem valores mais naturais
  bias_initializer = initializers.Zeros() # inicia o bias como 0
))

model.add(layers.Dropout(0.2)) # diz que 20% dos neurônios serão desligados para evitar overfitting
model.add(layers.Activation(activations.relu)) # filtra se o neurônio é importante (relu mantem os positivos e zera os negativos)

model.add(layers.Dense(64,
  kernel_regularizer = regularizers.L2(1e-4), 
  kernel_initializer = initializers.RandomNormal(stddev=1),
  bias_initializer = initializers.Zeros() 
))

model.add(layers.Activation(activations.relu))

model.add(layers.BatchNormalization()) # normaliza os valores de saída da camada para reduzir overfitting, melhorar desempenho.

model.add(layers.Dense(64,
  kernel_regularizer = regularizers.L2(1e-4), 
  kernel_initializer = initializers.RandomNormal(stddev=1),
  bias_initializer = initializers.Zeros() 
))

model.add(layers.Activation(activations.relu))

model.add(layers.Dense(2,
  kernel_regularizer = regularizers.L2(1e-4), 
  kernel_initializer = initializers.RandomNormal(stddev=1),
  bias_initializer = initializers.Zeros() 
))

model.add(layers.Activation(activations.softmax))

model.compile(
  optimizer=optimizers.Adam(), # Adam e SGD são os optimizers mais utilizados
  loss=losses.BinaryCrossentropy(),
  metrics=[metrics.Accuracy(), metrics.Precision()]
)

dataGen = image.ImageDataGenerator(
  rescale = 1.0/255, # redimensiona a escala de 0 a 255 para 0 a 1
  shear_range = 0.2,
  zoom_range = 0.2,
  horizontal_flip = True,
  vertical_flip = False,
  validation_split = 0.2
)

X_train = dataGen.flow_from_directory(
  'caminho/para/fotos',
  target_size = (64, 64), # tamanho desejado da imagem, redimensiona.
  batch_size = 32, # passa a quantidade de imagens desejadas para treino
  class_mode = 'categorical', # binarico, categorico e de regressão.
  subset = 'training'
)

X_test = dataGen.flow_from_directory(
  'caminho/para/fotos',
  target_size = (64, 64), # tamanho desejado da imagem, redimensiona.
  batch_size = 32, # passa a quantidade de imagens desejadas para treino
  class_mode = 'categorical', # binarico, categorico e de regressão.
  subset = 'validation'
)

model.fit(
  X_train, 
  steps_per_epoch = 1000, # steps_per_epoch * batch_size == len(dataset) <- recomendado
  epochs = 50,
  validation_steps = 100, # steps_per_epoch / 10 <- recomendado
  callbacks = [
    callbacks.EarlyStopping(patience = 4),
    callbacks.ModelCheckpoint(
      filepath = 'model.{epoch:02d}-{val_loss:.2f}.h5'
    )
  ]
) 

model.save('model')

FileNotFoundError: [WinError 3] O sistema não pode encontrar o caminho especificado: 'caminho/para/fotos'